# Testing Syntax Analysis 

Analysis tests...

In [ ]:
%load_ext nb_unittest

In [ ]:
"""@solution"""

a = 1 
b = 2 
a + b 

def c():
    """c"""
    print("c()")
    d = 1
    _ = a < b
    def e(f, g):
        """e"""
        h = 2
        type((d,f,g,h,))
        _ = a > b
    

In [ ]:
import nb_unittest 
import ast 

s = nb_unittest.get('@solution')

assert ast.Add in s.tokens
assert 'a' in s.assignments
assert 'b' in s.assignments
assert 'c' not in s.assignments
assert 'g' not in s.assignments
assert 'print' not in s.calls
assert 'type' not in s.calls 

# Ensure extracted tokens still contains a function def.
assert ast.FunctionDef in s.tokens

assert 'c' in s.functions

# Ensure we can see all tokens in "c"
assert ast.FunctionDef in s.functions['c'].tokens
assert ast.Lt in s.functions['c'].tokens
assert ast.Gt not in s.functions['c'].tokens

assert 'c' == s.functions['c'].docstring
assert 'd' in s.functions['c'].assignments
assert 'print' in s.functions['c'].calls
assert 'e' in s.functions['c'].functions
assert 'e' == s.functions['c'].functions["e"].docstring

assert 'h' in s.functions['c'].functions['e'].assignments
assert 'type' in s.functions['c'].functions['e'].calls
assert 'f' in s.functions['c'].functions['e'].arguments
assert 'g' in s.functions['c'].functions['e'].arguments

# Ensure we can see tokens in e
assert ast.Lt not in s.functions['c'].functions['e'].tokens
assert ast.Gt in s.functions['c'].functions['e'].tokens


In [ ]:
"""@solution2"""

_ = 1 < 2 

class A:
    """A"""
    B = 1
    C = 2
    _ = 1 > 2 

    class D:
        """D"""
        E = 3
        F = 4
        _ = 1 <= 2 
    
        def g(self,h,i):
            """g"""
            j = A.B + A.C + self.E + self.F + h + i
            type(j)
            _ = 1 >= 2 
    
    def k(self, l, m, *n, o, p, **q):
        """k"""
        print(5+l)
        _ = 1 % 2

In [ ]:
import nb_unittest 
import ast 

s = nb_unittest.get('@solution2')

assert "A" in s.classes
assert "print" not in s.calls
assert "type" not in s.calls

# FIX: make the tokens attribute work on scope like the others
assert ast.Add not in s.tokens
assert ast.ClassDef in s.tokens

assert "A" == s.classes['A'].docstring
assert "B" in s.classes['A'].assignments
assert "C" in s.classes['A'].assignments
assert "k" in s.classes['A'].functions
assert "k" == s.classes['A'].functions["k"].docstring

assert "D" in s.classes['A'].classes
assert "D" == s.classes['A'].classes["D"].docstring
assert "E" in s.classes['A'].classes["D"].assignments
assert "F" in s.classes['A'].classes["D"].assignments

assert "g" in s.classes['A'].classes["D"].functions
assert "g" == s.classes['A'].classes["D"].functions["g"].docstring
assert "j" in s.classes['A'].classes["D"].functions["g"].assignments
assert "type" in s.classes['A'].classes["D"].functions["g"].calls
assert "self" in s.classes['A'].classes["D"].functions["g"].arguments
assert "h" in s.classes['A'].classes["D"].functions["g"].arguments
assert "i" in s.classes['A'].classes["D"].functions["g"].arguments

# Ensure we can see the right tokens. Check the fix for the tokens bug. 
assert ast.Lt in s.tokens
assert ast.Gt not in s.tokens
assert ast.GtE not in s.tokens 
assert ast.LtE not in s.tokens
assert ast.Mod not in s.tokens

assert ast.Lt not in s.classes["A"].tokens
assert ast.Gt in s.classes["A"].tokens
assert ast.GtE not in s.classes["A"].tokens
assert ast.LtE not in s.classes["A"].tokens
assert ast.Mod not in s.classes["A"].tokens

assert ast.Lt not in s.classes["A"].classes["D"].tokens
assert ast.Gt not in s.classes["A"].classes["D"].tokens
assert ast.GtE not in s.classes["A"].classes["D"].tokens
assert ast.LtE in s.classes["A"].classes["D"].tokens
assert ast.Mod not in s.classes["A"].classes["D"].tokens

assert ast.Lt not in s.classes["A"].classes["D"].functions["g"].tokens
assert ast.Gt not in s.classes["A"].classes["D"].functions["g"].tokens
assert ast.GtE in s.classes["A"].classes["D"].functions["g"].tokens
assert ast.LtE not in s.classes["A"].classes["D"].functions["g"].tokens
assert ast.Mod not in s.classes["A"].classes["D"].functions["g"].tokens

assert ast.Lt not in s.classes["A"].functions["k"].tokens
assert ast.Gt not in s.classes["A"].functions["k"].tokens
assert ast.GtE not in s.classes["A"].functions["k"].tokens
assert ast.LtE not in s.classes["A"].functions["k"].tokens
assert ast.Mod in s.classes["A"].functions["k"].tokens

In [ ]:
## New: Test argument ordering. 
assert "l" == s.classes['A'].functions["k"].arguments[1]
assert "m" == s.classes['A'].functions["k"].arguments[2]
assert "*n" == s.classes['A'].functions["k"].arguments[3]
assert "o" == s.classes['A'].functions["k"].arguments[4]
assert "p" == s.classes['A'].functions["k"].arguments[5]
assert "**q" == s.classes['A'].functions["k"].arguments[6]

In [ ]:
"""@callincall"""

## Bugfix: Make sure we see calls in calls.
print(type(1))

In [ ]:
import nb_unittest 

c = nb_unittest.get('@callincall')
assert "print" in c.calls
assert "type" in c.calls

## Testing the New References 

A list of all referenced symbols.

In [ ]:
a = 100
b = 200

In [ ]:
"""@refs"""
c = 300 
print(a, b)

def foo():
    print(c)

In [ ]:
refs = nb_unittest.get("@refs")
assert {'a', 'b', 'print'} == refs.references
assert {'c', 'print'} == refs.functions["foo"].references

## Testing the Counting Syntax

Get counts of things where that might help. 


In [ ]:
a = 100
b = 200

In [ ]:
"""@counts"""
c = 300 
print(a, b)
print(c, a)
type(a)

def foo():
    print(c)

In [ ]:
counts = nb_unittest.get("@counts")
assert 2 == counts.count_calls("print")
assert 1 == counts.count_calls("type")
assert 0 == counts.count_calls("bogus")

assert 3 == counts.count_references('a')
assert 1 == counts.count_references('b')
assert 1 == counts.count_references('c')
assert 0 == counts.count_references('d')

assert 1 == counts.count_assignments('c')
assert 0 == counts.count_assignments('a')

## Testing Imports 



In [ ]:
"""@imp"""

import re 
import subprocess as foo
from math import pi

def a():
    import json, io

In [ ]:
imp = nb_unittest.get("@imp")

assert {"re", "subprocess", "math"} == imp.imports 
assert {"json", "io"} == imp.functions["a"].imports 